In [1]:
# Import required libraries for data processing, optimization, and serialization
try:
    import pandas as pd
    import numpy as np
    from scipy.optimize import linear_sum_assignment
    import random
    from collections import defaultdict
    import joblib
    import os
    print("All libraries imported successfully")
except ImportError as e:
    print(f"Error importing libraries: {e}")
    print("Please install required libraries using: !pip install pandas numpy scipy joblib")

All libraries imported successfully


In [2]:
# Load courses from CSV file
try:
    if not os.path.exists('courses.csv'):
        raise FileNotFoundError("courses.csv not found. Please run Cell 0 to create it.")
    courses_df = pd.read_csv('courses.csv')
    courses = courses_df.to_dict(orient='records')
    course_names = [c['course'] for c in courses]
    print(f"Loaded {len(course_names)} courses from courses.csv")
except Exception as e:
    print(f"Error loading courses.csv: {e}")

Loaded 49 courses from courses.csv


In [3]:
# Define common O'Level subjects (all possible subjects)
common_subjects = [
    'English Language', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'Agricultural Science', 'Geography', 'Economics',
    'Further Mathematics', 'Statistics', 'Fine Art', 'Technical Drawing', 'Introduction to Building Construction',
    'Bricklaying/Blocklaying', 'Concreting', 'Joinery', 'Carpentry', 'Decorative Painting', 'Ceramics', 'Graphics Design',
    'Graphic Printing', 'Basic Electricity', 'Introduction to Agricultural Science', 'Materials and Workshop Process and Machining',
    'Tractor Layout Power Unit Under Carriage and Auto Electricity', 'Basic Catering and Food Services', 'Bakery and Confectionaries',
    'Hotel & Catering Craft (Cookery)', 'Government', 'Commerce', 'Accounting', 'Literature in English', 'History', 'CRK', 'IRK', 'Social Studies'
]

# Map O'Level grades to numeric values (A1=1, F9=9)
grade_map = {'A1': 1, 'B2': 2, 'B3': 3, 'C4': 4, 'C5': 5, 'C6': 6, 'D7': 7, 'E8': 8, 'F9': 9}

# Define interest-to-course group mapping for interest-based scoring
course_groups = {
    "agriculture": ["Agric Extension & Communication Technology", "Agricultural Engineering", "Agriculture Resource Economics", "Animal Production & Health Services", "Crop Soil & Pest Management", "Ecotourism & Wildlife Management", "Fisheries & Aquaculture", "Food Science & Technology", "Forestry & Wood Technology"],
    "engineering": ["Agricultural Engineering", "Civil Engineering", "Computer Engineering", "Electrical / Electronics Engineering", "Industrial & Production Engineering", "Mechanical Engineering", "Metallurgical & Materials Engineering", "Mining Engineering"],
    "science": ["Applied Geology", "Applied Geophysics", "Biochemistry", "Biology", "Biomedical Technology", "Biotechnology", "Industrial Chemistry", "Industrial Mathematics", "Marine Science & Technology", "Mathematics", "Meteorology", "Microbiology", "Physics", "Statistics"],
    "technology": ["Architecture", "Building", "Computer Science", "Cyber Security", "Information & Communication Technology", "Information Systems", "Information Technology", "Software Engineering"],
    "health": ["Human Anatomy", "Medical Laboratory Science", "Physiology"],
    "management": ["Entrepreneurship", "Estate Management", "Quantity Surveying", "Surveying & Geoinformatics", "Urban & Regional Planning"],
    "design": ["Industrial Design", "Textile Design Technology"],
    "geoscience": ["Remote Sensing & Geoscience Information System"],
}

# Function to get related courses based on applicant's top interest
def get_related_courses(interest):
    try:
        for group, courses in course_groups.items():
            if interest in courses:
                return courses
        return []
    except Exception as e:
        print(f"Error in get_related_courses: {e}")
        return []

In [4]:
# Parsed eligibility requirements for all courses (derived from courses.csv)
parsed_req = {
    "Agric Extension & Communication Technology": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {1: ['Physics', 'Geography', 'Economics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Agricultural Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Introduction to Agricultural Science', 'Materials and Workshop Process and Machining', 'Tractor Layout Power Unit Under Carriage and Auto Electricity']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Agriculture Resource Economics": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics', 'Biology'],
        'or_groups': [],
        'optional': {1: ['Physics', 'Economics', 'Further Mathematics', 'Statistics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Animal Production & Health Services": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics', 'Physics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Applied Geology": {
        'mandatory': ['English Language', 'Biology', 'Chemistry', 'Mathematics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Applied Geophysics": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [['Chemistry', 'Biology']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Architecture": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Fine Art', 'Geography', 'Wood Work', 'Biology', 'Economics', 'Technical Drawing', 'Further Mathematics', 'Introduction to Building Construction', 'Bricklaying/Blocklaying', 'Concreting', 'Joinery', 'Carpentry', 'Decorative Painting', 'Ceramics', 'Graphics Design', 'Graphic Printing', 'Basic Electricity']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Biochemistry": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics', 'Physics', 'Biology'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Biology": {
        'mandatory': ['English Language', 'Biology', 'Chemistry'],
        'or_groups': [['Mathematics', 'Physics']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Biomedical Technology": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry', 'Biology'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Biotechnology": {
        'mandatory': ['English Language', 'Mathematics', 'Biology', 'Chemistry', 'Physics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Building": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Geography', 'Economics', 'Arts', 'Technical Drawing']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Civil Engineering": {
        'mandatory': ['English Language', 'Physics', 'Chemistry', 'Mathematics'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Further Mathematics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Computer Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Further Mathematics', 'Chemistry', 'Physics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Computer Science": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Biology', 'Chemistry', 'Agricultural Science', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Crop Soil & Pest Management": {
        'mandatory': ['Chemistry', 'Mathematics', 'Physics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {1: ['Physics', 'Geography', 'Economics', 'Government', 'Further Mathematics']},
        'thresholds': {'Physics': 8},
        'required_credit_count': 4
    },
    "Cyber Security": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Biology', 'Chemistry', 'Agricultural Science', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Ecotourism & Wildlife Management": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {2: ['Chemistry', 'Geography', 'Economics']},
        'thresholds': {'Physics': 8},
        'required_credit_count': 4
    },
    "Electrical / Electronics Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Further Mathematics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Entrepreneurship": {
        'mandatory': ['English Language', 'Economics', 'Mathematics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Estate Management": {
        'mandatory': ['English Language', 'Mathematics', 'Chemistry'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Fisheries & Aquaculture": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {1: ['Physics', 'Geography', 'Economics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Food Science & Technology": {
        'mandatory': ['English Language', 'Mathematics', 'Chemistry'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {1: ['Physics', 'Basic Catering and Food Services', 'Bakery and Confectionaries', 'Hotel & Catering Craft (Cookery)']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Forestry & Wood Technology": {
        'mandatory': ['English Language', 'Chemistry', 'Mathematics', 'Physics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {},
        'thresholds': {'Physics': 8},
        'required_credit_count': 4
    },
    "Human Anatomy": {
        'mandatory': ['English Language', 'Mathematics', 'Biology', 'Chemistry', 'Physics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Industrial & Production Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Geography', 'Further Mathematics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Industrial Chemistry": {
        'mandatory': ['English Language', 'Mathematics', 'Chemistry', 'Physics'],
        'or_groups': [['Biology', 'Agricultural Science']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Industrial Design": {
        'mandatory': ['English Language', 'Fine Art', 'Mathematics', 'Chemistry'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Industrial Mathematics": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Chemistry', 'Economics', 'Biology', 'Agricultural Science']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Information & Communication Technology": {
        'mandatory': ['English Language', 'Economics', 'Mathematics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Information Systems": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Biology', 'Chemistry', 'Agricultural Science', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Information Technology": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Biology', 'Chemistry', 'Agricultural Science', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Marine Science & Technology": {
        'mandatory': ['English Language', 'Mathematics', 'Biology'],
        'or_groups': [],
        'optional': {2: ['Physics', 'Chemistry', 'Mathematics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Mathematics": {
        'mandatory': ['English Language', 'Mathematics'],
        'or_groups': [['Physics', 'Chemistry']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Mechanical Engineering": {
        'mandatory': ['English Language', 'Physics', 'Chemistry', 'Mathematics'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Further Mathematics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Medical Laboratory Science": {
        'mandatory': ['English Language', 'Mathematics', 'Chemistry', 'Biology', 'Physics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Metallurgical & Materials Engineering": {
        'mandatory': ['English Language', 'Physics', 'Chemistry', 'Mathematics'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Geography', 'Further Mathematics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Meteorology": {
        'mandatory': ['English Language', 'Physics', 'Mathematics'],
        'or_groups': [['Chemistry', 'Geography']],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Microbiology": {
        'mandatory': ['English Language', 'Mathematics', 'Chemistry', 'Biology', 'Physics'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Mining Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Biology', 'Geography', 'Further Mathematics']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Physics": {
        'mandatory': ['English Language', 'Physics', 'Chemistry', 'Mathematics'],
        'or_groups': [],
        'optional': {1: ['Further Mathematics', 'Biology', 'Agricultural Science']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Physiology": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry', 'Biology'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Quantity Surveying": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Fine Art', 'Geography', 'Wood Work', 'Biology', 'Economics', 'Technical Drawing', 'Further Mathematics', 'Introduction to Building Construction', 'Bricklaying/Blocklaying', 'Concreting', 'Joinery', 'Carpentry', 'Decorative Painting', 'Ceramics', 'Graphics Design', 'Graphic Printing', 'Basic Electricity']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Remote Sensing & Geoscience Information System": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Fine Art', 'Geography', 'Wood Work', 'Biology', 'Economics', 'Technical Drawing', 'Further Mathematics', 'Introduction to Building Construction', 'Bricklaying/Blocklaying', 'Concreting', 'Joinery', 'Carpentry', 'Decorative Painting', 'Ceramics', 'Graphics Design', 'Graphic Printing', 'Basic Electricity']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Software Engineering": {
        'mandatory': ['English Language', 'Mathematics', 'Physics'],
        'or_groups': [],
        'optional': {2: ['Biology', 'Chemistry', 'Agricultural Science', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Statistics": {
        'mandatory': ['English Language', 'Mathematics'],
        'or_groups': [],
        'optional': {3: ['Physics', 'Statistics', 'Chemistry', 'Further Mathematics', 'Economics', 'Geography']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Surveying & Geoinformatics": {
        'mandatory': ['English Language', 'Mathematics', 'Physics', 'Chemistry'],
        'or_groups': [],
        'optional': {1: ['Fine Art', 'Geography', 'Biology', 'Economics', 'Technical Drawing', 'Further Mathematics', 'Introduction to Building Construction', 'Bricklaying/Blocklaying', 'Concreting', 'Joinery', 'Carpentry', 'Decorative Painting', 'Ceramics', 'Graphics Design', 'Graphic Printing', 'Basic Electricity']},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Textile Design Technology": {
        'mandatory': ['English Language', 'Fine Art'],
        'or_groups': [],
        'optional': {},
        'thresholds': {},
        'required_credit_count': 5
    },
    "Urban & Regional Planning": {
        'mandatory': ['English Language', 'Mathematics', 'Geography'],
        'or_groups': [],
        'optional': {2: ['Physics', 'Chemistry', 'Economics', 'Government', 'Biology', 'Art', 'History', 'IRK', 'CRK', 'Social Studies', 'Technical Drawing']},
        'thresholds': {},
        'required_credit_count': 5
    }
}

# Validate parsed_req against course_names
try:
    missing_courses = [c for c in course_names if c not in parsed_req]
    if missing_courses:
        print(f"Warning: The following courses in courses.csv are missing in parsed_req: {missing_courses}")
    else:
        print("All courses in courses.csv have corresponding parsed_req entries")
except NameError:
    print("Error: course_names not defined. Please run Cell 2 first.")

All courses in courses.csv have corresponding parsed_req entries


In [5]:
# Parse O'Level subjects and grades from applicant data
def parse_applicant_olevel(row):
    """
    Parse O'Level subject:grade strings into a dictionary of subject:grade (numeric).
    Input: DataFrame row with subject1 to subject5 columns (e.g., 'Mathematics:C4')
    Output: Dictionary of subject:numeric_grade (e.g., {'Mathematics': 4})
    """
    try:
        subjects = {}
        for i in range(1, 6):
            key = f'subject{i}'
            if key in row and pd.notnull(row[key]):
                sub_grade = row[key]
                if ':' in sub_grade:
                    sub, grade = sub_grade.split(':')
                    sub = sub.strip()
                    grade = grade.strip()
                    if sub in common_subjects and grade in grade_map:
                        subjects[sub] = grade_map[grade]
                    else:
                        print(f"Warning: Invalid subject ({sub}) or grade ({grade}) for applicant {row.get('applicant_id', 'unknown')}")
        return subjects
    except Exception as e:
        print(f"Error in parse_applicant_olevel for applicant {row.get('applicant_id', 'unknown')}: {e}")
        return {}

# Check if applicant is eligible for a course
def is_eligible(applicant_subjects, course):
    """
    Check if applicant meets O'Level requirements for a course.
    Input: applicant_subjects (dict), course (str)
    Output: Boolean indicating eligibility
    """
    try:
        if course not in parsed_req:
            print(f"Error: Course {course} not found in parsed_req")
            return False
        p = parsed_req[course]
        thresh = p['thresholds']
        good_credit = {s for s in applicant_subjects if applicant_subjects[s] <= 6}
        if len(good_credit) < p['required_credit_count']:
            return False
        for s in p['mandatory']:
            if s not in applicant_subjects or applicant_subjects[s] > thresh.get(s, 6):
                return False
        for group in p['or_groups']:
            if not any(sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6) for sub in group):
                return False
        for num, group in p['optional'].items():
            count = sum(1 for sub in group if sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6))
            if count < num:
                return False
        return True
    except Exception as e:
        print(f"Error in is_eligible for course {course}: {e}")
        return False

# Compute sum of grades and count of satisfied subjects for scoring
def compute_grade_sum(applicant_subjects, course):
    """
    Compute sum of grades and count of subjects meeting course requirements.
    Input: applicant_subjects (dict), course (str)
    Output: Tuple (grade_sum, count)
    """
    try:
        p = parsed_req[course]
        thresh = p['thresholds']
        grade_sum = 0
        count = 0
        # Mandatory subjects
        for s in p['mandatory']:
            if s in applicant_subjects and applicant_subjects[s] <= thresh.get(s, 6):
                grade_sum += applicant_subjects[s]
                count += 1
        # OR groups (take min grade)
        for group in p['or_groups']:
            candidates = [applicant_subjects[sub] for sub in group if sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6)]
            if candidates:
                grade_sum += min(candidates)
                count += 1
        # Optional subjects (take lowest num grades)
        for num, group in p['optional'].items():
            candidates = [applicant_subjects[sub] for sub in group if sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6)]
            candidates.sort()
            if len(candidates) >= num:
                grade_sum += sum(candidates[:num])
                count += num
        return grade_sum, count
    except Exception as e:
        print(f"Error in compute_grade_sum for course {course}: {e}")
        return 0, 0

# Compute applicant score for a course
def compute_score(utme_score, grade_sum, count, interest_match, bonus):
    """
    Compute score based on UTME, O'Level grades, and interest match.
    Input: utme_score (int), grade_sum (int), count (int), interest_match (bool), bonus (float)
    Output: Float score (0 to 1+bonus)
    """
    try:
        if count == 0:
            return 0
        normalized_utme = utme_score / 400  # Normalize UTME score (0-400)
        average_grade = grade_sum / count
        normalized_grade = (9 - average_grade) / 8  # Normalize grades (lower is better)
        score = 0.5 * normalized_utme + 0.5 * normalized_grade
        if interest_match:
            score += bonus
        return max(0, min(score, 1 + bonus))  # Ensure score is within valid range
    except Exception as e:
        print(f"Error in compute_score: {e}")
        return 0

In [6]:
# Function to generate synthetic applicant data for testing
def generate_synthetic_data(num_applicants=1000):
    """
    Generate synthetic applicant data with UTME scores, O'Level subjects, and top interest.
    Input: num_applicants (int)
    Output: DataFrame with applicant_id, utme_score, top_interest, subject1 to subject5
    """
    try:
        data = []
        grades = list(grade_map.keys())
        for id in range(1, num_applicants + 1):
            utme = random.randint(180, 400)  # Random UTME score
            subjects = random.sample(common_subjects, 5)  # Random 5 subjects
            subject_grades = [random.choice(grades) for _ in range(5)]  # Random grades
            top_interest = random.choice(course_names)  # Random course interest
            row = {'applicant_id': id, 'utme_score': utme, 'top_interest': top_interest}
            for i in range(5):
                row[f'subject{i+1}'] = f"{subjects[i]}:{subject_grades[i]}"
            data.append(row)
        df = pd.DataFrame(data)
        df.to_csv('applicants.csv', index=False)
        return df
    except Exception as e:
        print(f"Error generating synthetic data: {e}")
        return pd.DataFrame()

# Load or generate applicant data
try:
    # Uncomment the line below to load real data instead of synthetic
    # applicants = pd.read_csv('your_real_applicants.csv')
    applicants = generate_synthetic_data(1000)
    if applicants.empty:
        raise ValueError("No applicant data generated or loaded")
    print(f"Loaded/generated {len(applicants)} applicants")
    display(applicants.head())  # Show first 5 rows
except Exception as e:
    print(f"Error loading/generating applicants: {e}")

Loaded/generated 1000 applicants


,applicant_id,utme_score,top_interest,subject1,subject2,subject3,subject4,subject5
0,1,342,Metallurgical & Materials Engineering,Joinery:C4,Carpentry:F9,IRK:D7,Ceramics:C6,Mathematics:C6
1,2,261,Surveying & Geoinformatics,Tractor Layout Power Unit Under Carriage and A...,CRK:E8,Basic Catering and Food Services:D7,Geography:E8,Concreting:C5
2,3,389,Statistics,IRK:C5,Commerce:F9,Graphic Printing:B3,Joinery:F9,Basic Electricity:A1
3,4,371,Animal Production & Health Services,Concreting:F9,Biology:D7,Graphics Design:C4,Basic Catering and Food Services:C4,Joinery:F9
4,5,384,Mechanical Engineering,Geography:C4,Basic Electricity:E8,CRK:F9,Joinery:D7,Carpentry:E8


In [ ]:
# Configurable parameters
interest_bonus = 0.1  # 10% score bonus for interest match

# Program capacities for all courses
capacities = {
    "Agric Extension & Communication Technology": 100,
    "Agricultural Engineering": 100,
    "Agriculture Resource Economics": 100,
    "Animal Production & Health Services": 100,
    "Applied Geology": 100,
    "Applied Geophysics": 100,
    "Architecture": 100,
    "Biochemistry": 100,
    "Biology": 100,
    "Biomedical Technology": 100,
    "Biotechnology": 100,
    "Building": 100,
    "Civil Engineering": 150,
    "Computer Engineering": 100,
    "Computer Science": 200,
    "Crop Soil & Pest Management": 100,
    "Cyber Security": 100,
    "Ecotourism & Wildlife Management": 100,
    "Electrical / Electronics Engineering": 100,
    "Entrepreneurship": 100,
    "Estate Management": 100,
    "Fisheries & Aquaculture": 100,
    "Food Science & Technology": 100,
    "Forestry & Wood Technology": 100,
    "Human Anatomy": 50,
    "Industrial & Production Engineering": 100,
    "Industrial Chemistry": 100,
    "Industrial Design": 100,
    "Industrial Mathematics": 100,
    "Information & Communication Technology": 100,
    "Information Systems": 100,
    "Information Technology": 100,
    "Marine Science & Technology": 100,
    "Mathematics": 100,
    "Mechanical Engineering": 100,
    "Medical Laboratory Science": 50,
    "Metallurgical & Materials Engineering": 100,
    "Meteorology": 100,
    "Microbiology": 100,
    "Mining Engineering": 100,
    "Physics": 100,
    "Physiology": 50,
    "Quantity Surveying": 100,
    "Remote Sensing & Geoscience Information System": 100,
    "Software Engineering": 100,
    "Statistics": 100,
    "Surveying & Geoinformatics": 100,
    "Textile Design Technology": 100,
    "Urban & Regional Planning": 100
}

# Course-specific UTME cut-off marks
cutoff_marks = {
    "Human Anatomy": 250,
    "Medical Laboratory Science": 250,
    "Physiology": 250,
    "Civil Engineering": 220,
    "Computer Engineering": 220,
    "Electrical / Electronics Engineering": 220,
    "Mechanical Engineering": 220,
    "Metallurgical & Materials Engineering": 220,
    "Mining Engineering": 220,
    "Computer Science": 230,
    "Cyber Security": 230,
    "Software Engineering": 230
}
# Set default cut-off of 180 for all other courses
for course in course_names:
    if course not in cutoff_marks:
        cutoff_marks[course] = 180

# Validate capacities and cut-off marks
try:
    missing_capacities = [c for c in course_names if c not in capacities]
    missing_cutoffs = [c for c in course_names if c not in cutoff_marks]
    if missing_capacities:
        print(f"Warning: The following courses are missing capacities: {missing_capacities}")
    else:
        print("All courses have defined capacities")
    if missing_cutoffs:
        print(f"Warning: The following courses are missing cut-off marks: {missing_cutoffs}")
    else:
        print("All courses have defined cut-off marks")
except NameError:
    print("Error: course_names not defined. Please run Cell 2 first.")

# Preprocess O'Level subjects
try:
    applicants['subjects'] = applicants.apply(parse_applicant_olevel, axis=1)
except Exception as e:
    print(f"Error preprocessing applicant subjects: {e}")

# Compute eligibility and scores for each applicant-course pair
scores = []
try:
    for i, row in applicants.iterrows():
        applicant_id = row['applicant_id']
        utme = row['utme_score']
        subjects = row['subjects']
        top_interest = row['top_interest']
        related = get_related_courses(top_interest)
        for course in course_names:
            # Check eligibility, including UTME cut-off mark
            eligible = is_eligible(subjects, course) and utme >= cutoff_marks[course]
            score = 0
            if eligible:
                grade_sum, count = compute_grade_sum(subjects, course)
                interest_match = course == top_interest or course in related
                score = compute_score(utme, grade_sum, count, interest_match, interest_bonus)
            scores.append({'applicant_id': applicant_id, 'course': course, 'eligible': eligible, 'score': score})
    scores_df = pd.DataFrame(scores)
    scores_df.to_csv('scores.csv', index=False)
    print(f"Saved scores.csv with {len(scores_df)} entries")
    display(scores_df.head())  # Show first 5 rows
except Exception as e:
    print(f"Error computing scores: {e}")

In [ ]:
# Allocate applicants to programs
try:
    remaining_capacity = capacities.copy()
    assignments = []
    assigned = set()

    # Step 1: Assign to top interest if eligible and capacity available
    for course in course_names:
        if course not in remaining_capacity:
            continue
        candidates = []
        for _, row in applicants[applicants['top_interest'] == course].iterrows():
            a = row['applicant_id']
            sc_row = scores_df[(scores_df['applicant_id'] == a) & (scores_df['course'] == course)]
            if not sc_row.empty and sc_row['eligible'].item():
                score = sc_row['score'].item()
                utme = row['utme_score']
                grade_sum, _ = compute_grade_sum(row['subjects'], course)
                candidates.append((a, score, utme, -grade_sum))  # Negative grade_sum for tie-breaking
        if candidates:
            candidates.sort(key=lambda x: (-x[1], -x[2], x[3]))  # Sort by score desc, utme desc, grade_sum asc
            k = min(len(candidates), remaining_capacity[course])
            for rank, (a, _, _, _) in enumerate(candidates[:k], 1):
                assignments.append({'applicant_id': a, 'assigned_program': course, 'assigned_rank': rank, 'reason': 'first_choice_qualified'})
                assigned.add(a)
            remaining_capacity[course] -= k

    # Step 2: Assign remaining applicants using Hungarian algorithm
    remaining_applicants = list(set(applicants['applicant_id']) - assigned)
    if remaining_applicants:
        course_seats = []
        for course, cap in remaining_capacity.items():
            course_seats += [course] * cap
        K = len(course_seats)
        if K > 0:
            N = len(remaining_applicants)
            cost_matrix = np.full((N, K), -1000.0)  # Negative for maximization
            for col, course in enumerate(course_seats):
                for i, a in enumerate(remaining_applicants):
                    sc_row = scores_df[(scores_df['applicant_id'] == a) & (scores_df['course'] == course)]
                    if not sc_row.empty and sc_row['eligible'].item():
                        score = sc_row['score'].item()
                        cost_matrix[i, col] = -score
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            temp_assign = defaultdict(list)
            for i, j in zip(row_ind, col_ind):
                if cost_matrix[i, j] > -500:  # Valid assignment threshold
                    a = remaining_applicants[i]
                    course = course_seats[j]
                    score = -cost_matrix[i, j]
                    utme = applicants[applicants['applicant_id'] == a]['utme_score'].item()
                    grade_sum, _ = compute_grade_sum(applicants[applicants['applicant_id'] == a]['subjects'].item(), course)
                    temp_assign[course].append((a, score, utme, -grade_sum))
            for course, cand in temp_assign.items():
                cand.sort(key=lambda x: (-x[1], -x[2], x[3]))
                for rank, (a, _, _, _) in enumerate(cand, 1):
                    top_interest = applicants[applicants['applicant_id'] == a]['top_interest'].item()
                    reason = 'assigned_alternative_matched_interest' if course in get_related_courses(top_interest) else 'assigned_alternative'
                    assignments.append({'applicant_id': a, 'assigned_program': course, 'assigned_rank': rank, 'reason': reason})

    # Step 3: Mark unassigned applicants
    all_assigned = {ass['applicant_id'] for ass in assignments}
    for a in applicants['applicant_id']:
        if a not in all_assigned:
            assignments.append({'applicant_id': a, 'assigned_program': 'UNASSIGNED', 'assigned_rank': None, 'reason': 'no_eligible_seat'})

    # Save and display assignments
    assignments_df = pd.DataFrame(assignments)
    assignments_df.to_csv('assignments.csv', index=False)
    print(f"Saved assignments.csv with {len(assignments_df)} entries")
    display(assignments_df.head())  # Show first 5 rows
except Exception as e:
    print(f"Error during allocation: {e}")

In [ ]:
# Save the parsed requirements as the model
try:
    joblib.dump(parsed_req, 'model.joblib')
    print("Model saved as model.joblib")
except Exception as e:
    print(f"Error saving model: {e}")

In [ ]:
# Unit tests to verify system functionality
def test_parse_applicant_olevel():
    """Test parsing of O'Level subjects and grades"""
    row = {'applicant_id': 1, 'subject1': 'Mathematics:C4', 'subject2': 'English Language:A1', 'subject3': 'Physics:B3', 'subject4': 'Chemistry:C6', 'subject5': 'Biology:D7'}
    subjects = parse_applicant_olevel(row)
    assert subjects == {'Mathematics': 4, 'English Language': 1, 'Physics': 3, 'Chemistry': 6, 'Biology': 7}, "Failed parse_applicant_olevel"

def test_is_eligible():
    """Test eligibility checking for various courses"""
    subjects = {'English Language': 1, 'Mathematics': 4, 'Physics': 3, 'Chemistry': 6, 'Biology': 2}
    assert is_eligible(subjects, "Computer Science") == True, "Failed eligibility for Computer Science"
    subjects['Biology'] = 9
    assert is_eligible(subjects, "Human Anatomy") == False, "Failed ineligibility for Human Anatomy"
    subjects = {'English Language': 1, 'Fine Art': 3, 'Mathematics': 4}
    assert is_eligible(subjects, "Textile Design Technology") == True, "Failed eligibility for Textile Design Technology"

def test_compute_grade_sum():
    """Test grade sum and count computation"""
    subjects = {'English Language': 1, 'Mathematics': 4, 'Physics': 3, 'Chemistry': 6, 'Biology': 2}
    grade_sum, count = compute_grade_sum(subjects, "Biochemistry")
    assert count == 5 and grade_sum == 16, "Failed compute_grade_sum for Biochemistry"
    grade_sum, count = compute_grade_sum(subjects, "Crop Soil & Pest Management")
    assert count >= 4, "Failed compute_grade_sum for Crop Soil & Pest Management"

def test_compute_score():
    """Test score computation"""
    score = compute_score(utme_score=320, grade_sum=16, count=5, interest_match=True, bonus=0.1)
    assert abs(score - 0.85) < 0.1, "Failed compute_score"

# Run tests
try:
    test_parse_applicant_olevel()
    test_is_eligible()
    test_compute_grade_sum()
    test_compute_score()
    print("All unit tests passed successfully!")
except AssertionError as e:
    print(f"Unit test failed: {e}")
except Exception as e:
    print(f"Error running unit tests: {e}")

In [ ]:
# Import required libraries (already imported in Cell 1, but listed for clarity)
import pandas as pd
import os

# Load assignments and applicants data
try:
    if not os.path.exists('assignments.csv'):
        raise FileNotFoundError("assignments.csv not found. Please run Cell 8 first.")
    if not os.path.exists('applicants.csv'):
        raise FileNotFoundError("applicants.csv not found. Please run Cell 6 first.")
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    
    # Analysis 1: Number of students assigned per course
    assigned_counts = assignments_df[assignments_df['assigned_program'] != 'UNASSIGNED']['assigned_program'].value_counts()
    print("\nNumber of Students Assigned per Course:")
    print(assigned_counts)
    
    # Analysis 2: Number of unassigned applicants
    unassigned_count = len(assignments_df[assignments_df['assigned_program'] == 'UNASSIGNED'])
    print(f"\nNumber of Unassigned Applicants: {unassigned_count}")
    
    # Analysis 3: Success rate of first-choice placements
    merged_df = assignments_df.merge(applicants_df[['applicant_id', 'top_interest']], on='applicant_id', how='left')
    first_choice_success = len(merged_df[merged_df['assigned_program'] == merged_df['top_interest']])
    total_assigned = len(merged_df[merged_df['assigned_program'] != 'UNASSIGNED'])
    success_rate = (first_choice_success / total_assigned * 100) if total_assigned > 0 else 0
    print(f"\nFirst-Choice Placement Success Rate: {success_rate:.2f}% ({first_choice_success}/{total_assigned})")
    
    # Analysis 4: Capacity utilization
    capacity_utilization = {}
    for course, count in assigned_counts.items():
        capacity = capacities.get(course, 0)
        utilization = (count / capacity * 100) if capacity > 0 else 0
        capacity_utilization[course] = utilization
    print("\nCapacity Utilization (%):")
    for course, util in capacity_utilization.items():
        print(f"{course}: {util:.2f}%")
    
except Exception as e:
    print(f"Error analyzing allocations: {e}")

In [ ]:
```chartjs
{
  "type": "bar",
  "data": {
    "labels": [],
    "datasets": [{
      "label": "Number of Students Assigned",
      "data": [],
      "backgroundColor": "rgba(54, 162, 235, 0.8)",
      "borderColor": "rgba(54, 162, 235, 1)",
      "borderWidth": 1
    }]
  },
  "options": {
    "indexAxis": "y",
    "scales": {
      "x": {
        "beginAtZero": true,
        "title": {
          "display": true,
          "text": "Number of Students"
        }
      },
      "y": {
        "title": {
          "display": true,
          "text": "Course"
        }
      }
    },
    "plugins": {
      "title": {
        "display": true,
        "text": "Students Assigned per Course"
      },
      "legend": {
        "display": false
      }
    }
  }
}
```

# Populate the chart with data
try:
    if not os.path.exists('assignments.csv'):
        raise FileNotFoundError("assignments.csv not found. Please run Cell 8 first.")
    assignments_df = pd.read_csv('assignments.csv')
    assigned_counts = assignments_df[assignments_df['assigned_program'] != 'UNASSIGNED']['assigned_program'].value_counts().sort_index()
    courses = assigned_counts.index.tolist()
    counts = assigned_counts.values.tolist()
    
    # Update chart data
    chart_data = {
        "type": "bar",
        "data": {
            "labels": courses,
            "datasets": [{
                "label": "Number of Students Assigned",
                "data": counts,
                "backgroundColor": "rgba(54, 162, 235, 0.8)",
                "borderColor": "rgba(54, 162, 235, 1)",
                "borderWidth": 1
            }]
        },
        "options": {
            "indexAxis": "y",
            "scales": {
                "x": {
                    "beginAtZero": true,
                    "title": {
                        "display": true,
                        "text": "Number of Students"
                    }
                },
                "y": {
                    "title": {
                        "display": true,
                        "text": "Course"
                    }
                }
            },
            "plugins": {
                "title": {
                    "display": true,
                    "text": "Students Assigned per Course"
                },
                "legend": {
                    "display": false
                }
            }
        }
    }
    print("Bar chart created showing students assigned per course")
except Exception as e:
    print(f"Error creating visualization: {e}")

In [ ]:
# Function to predict placement for a new applicant
def predict_placement(utme_score, olevel_subjects, top_interest):
    """
    Predict the program an applicant is most likely to be assigned to.
    Input: utme_score (int), olevel_subjects (dict of subject:grade), top_interest (str)
    Output: Dictionary with predicted program, score, and eligibility details
    """
    try:
        if not isinstance(utme_score, (int, float)) or utme_score < 0 or utme_score > 400:
            raise ValueError("UTME score must be between 0 and 400")
        if top_interest not in course_names:
            raise ValueError(f"Top interest '{top_interest}' not in available courses")
        if not isinstance(olevel_subjects, dict):
            raise ValueError("O'Level subjects must be a dictionary of subject:grade")
        
        results = []
        related = get_related_courses(top_interest)
        for course in course_names:
            eligible = is_eligible(olevel_subjects, course) and utme_score >= cutoff_marks[course]
            score = 0
            if eligible:
                grade_sum, count = compute_grade_sum(olevel_subjects, course)
                interest_match = course == top_interest or course in related
                score = compute_score(utme_score, grade_sum, count, interest_match, interest_bonus)
            results.append({'course': course, 'eligible': eligible, 'score': score})
        
        results_df = pd.DataFrame(results)
        eligible_courses = results_df[results_df['eligible'] & (results_df['score'] > 0)]
        if eligible_courses.empty:
            return {'applicant_id': 'new_applicant', 'predicted_program': 'UNASSIGNED', 'score': 0, 'reason': 'no_eligible_course'}
        
        # Select the course with the highest score
        best_course = eligible_courses.loc[eligible_courses['score'].idxmax()]
        return {
            'applicant_id': 'new_applicant',
            'predicted_program': best_course['course'],
            'score': best_course['score'],
            'reason': 'highest_score' if best_course['course'] == top_interest else 'highest_score_alternative'
        }
    except Exception as e:
        print(f"Error in predict_placement: {e}")
        return {'applicant_id': 'new_applicant', 'predicted_program': 'UNASSIGNED', 'score': 0, 'reason': f'error: {str(e)}'}

# Example usage
try:
    example_applicant = {
        'utme_score': 260,
        'olevel_subjects': {
            'English Language': grade_map['A1'],
            'Mathematics': grade_map['B3'],
            'Physics': grade_map['C4'],
            'Chemistry': grade_map['C6'],
            'Biology': grade_map['B2']
        },
        'top_interest': 'Computer Science'
    }
    prediction = predict_placement(
        example_applicant['utme_score'],
        example_applicant['olevel_subjects'],
        example_applicant['top_interest']
    )
    print("\nExample Applicant Prediction:")
    print(f"Predicted Program: {prediction['predicted_program']}")
    print(f"Score: {prediction['score']:.2f}")
    print(f"Reason: {prediction['reason']}")
except Exception as e:
    print(f"Error running example prediction: {e}")

In [ ]:
# Generate a summary report in Markdown
try:
    from datetime import datetime
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    
    # Compute key metrics
    assigned_counts = assignments_df[assignments_df['assigned_program'] != 'UNASSIGNED']['assigned_program'].value_counts()
    unassigned_count = len(assignments_df[assignments_df['assigned_program'] == 'UNASSIGNED'])
    merged_df = assignments_df.merge(applicants_df[['applicant_id', 'top_interest']], on='applicant_id', how='left')
    first_choice_success = len(merged_df[merged_df['assigned_program'] == merged_df['top_interest']])
    total_assigned = len(merged_df[merged_df['assigned_program'] != 'UNASSIGNED'])
    success_rate = (first_choice_success / total_assigned * 100) if total_assigned > 0 else 0
    
    # Generate Markdown report
    report = f"""
# FUTA Admissions Placement Model Report
*Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*

## Overview
This report summarizes the results of the FUTA admissions placement model, which allocates applicants to programs based on UTME scores, O'Level grades, course-specific cut-off marks, and applicant interests.

## Key Metrics
- **Total Applicants**: {len(applicants_df)}
- **Total Assigned**: {total_assigned}
- **Total Unassigned**: {unassigned_count}
- **First-Choice Success Rate**: {success_rate:.2f}% ({first_choice_success}/{total_assigned})
- **Courses Processed**: {len(assigned_counts)}

## Allocation Summary
The following table shows the number of students assigned to each course:

| Course | Students Assigned | Capacity | Utilization (%) |
|--------|-------------------|----------|----------------|
"""
    for course, count in assigned_counts.items():
        capacity = capacities.get(course, 0)
        utilization = (count / capacity * 100) if capacity > 0 else 0
        report += f"| {course} | {count} | {capacity} | {utilization:.2f} |\n"
    
    report += """
## Methodology
- **Eligibility**: Applicants must meet O'Level subject requirements and course-specific UTME cut-off marks (e.g., 250 for medical courses, 230 for Computer Science, 180 for others).
- **Scoring**: Composite score based on normalized UTME score (50%), normalized O'Level grades (50%), and a {interest_bonus*100}% bonus for interest match.
- **Allocation**: Two-step process: (1) Assign first-choice qualified applicants, (2) Use Hungarian algorithm for remaining applicants.
- **Error Handling**: Comprehensive validation and error handling ensure robustness.
- **Testing**: Unit tests verify parsing, eligibility, and scoring functions.

## Files Generated
- **courses.csv**: Course requirements.
- **applicants.csv**: Applicant data (UTME scores, O'Level subjects, top interests).
- **scores.csv**: Eligibility and scores for applicant-course pairs.
- **assignments.csv**: Final allocations with ranks and reasons.
- **model.joblib**: Serialized eligibility rules.

## Recommendations
- Use real applicant data to validate the model.
- Adjust course capacities and cut-off marks based on official FUTA guidelines.
- Extend the model to include UTME subject validation for greater accuracy.

*End of Report*
"""
    
    # Save report to file
    with open('admissions_report.md', 'w') as f:
        f.write(report)
    print("Summary report saved as admissions_report.md")
    
    # Display report in notebook
    from IPython.display import Markdown
    display(Markdown(report))
except Exception as e:
    print(f"Error generating summary report: {e}")

In [ ]:
# Import required libraries (already imported in Cell 1)
import pandas as pd
import os

# Function to parse UTME subjects from applicant data
def parse_utme_subjects(row):
    """
    Parse UTME subjects from applicant data.
    Input: DataFrame row with utme_subjects (e.g., 'English,Mathematics,Physics,Chemistry')
    Output: List of subjects (e.g., ['English', 'Mathematics', 'Physics', 'Chemistry'])
    """
    try:
        if 'utme_subjects' not in row or pd.isna(row['utme_subjects']):
            return []
        return [s.strip() for s in row['utme_subjects'].split(',')]
    except Exception as e:
        print(f"Error parsing UTME subjects for applicant {row.get('applicant_id', 'unknown')}: {e}")
        return []

# Modified eligibility function to include UTME subject validation
def is_eligible_with_utme(applicant_subjects, utme_subjects, course):
    """
    Check if applicant meets O'Level and UTME subject requirements for a course.
    Input: applicant_subjects (dict), utme_subjects (list), course (str)
    Output: Boolean indicating eligibility
    """
    try:
        if course not in parsed_req or course not in courses_df['course'].values:
            print(f"Error: Course {course} not found")
            return False
        
        # Check O'Level requirements (from original is_eligible)
        thresh = parsed_req[course]['thresholds']
        good_credit = {s for s in applicant_subjects if applicant_subjects[s] <= 6}
        if len(good_credit) < parsed_req[course]['required_credit_count']:
            return False
        for s in parsed_req[course]['mandatory']:
            if s not in applicant_subjects or applicant_subjects[s] > thresh.get(s, 6):
                return False
        for group in parsed_req[course]['or_groups']:
            if not any(sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6) for sub in group):
                return False
        for num, group in parsed_req[course]['optional'].items():
            count = sum(1 for sub in group if sub in applicant_subjects and applicant_subjects[sub] <= thresh.get(sub, 6))
            if count < num:
                return False
        
        # Check UTME subjects
        course_row = courses_df[courses_df['course'] == course]
        if course_row.empty:
            return False
        required_utme = course_row['utme_subjects'].iloc[0].replace('Use of English', 'English').split(',')
        required_utme = [s.strip().split('/')[0] for s in required_utme]  # Handle alternatives (e.g., Biology/Agriculture)
        required_utme = [s for s in required_utme if s not in ['and', 'or', 'any of']]  # Clean up
        for req in required_utme:
            if req not in utme_subjects:
                return False
        return True
    except Exception as e:
        print(f"Error in is_eligible_with_utme for course {course}: {e}")
        return False

# Update applicants.csv with synthetic UTME subjects for testing
try:
    if not os.path.exists('applicants.csv'):
        raise FileNotFoundError("applicants.csv not found. Please run Cell 6 first.")
    applicants = pd.read_csv('applicants.csv')
    
    # Add synthetic UTME subjects (for testing)
    utme_subject_pool = ['English', 'Mathematics', 'Physics', 'Chemistry', 'Biology', 'Agricultural Science', 'Economics', 'Geography']
    applicants['utme_subjects'] = applicants.apply(
        lambda x: ','.join(['English'] + random.sample([s for s in utme_subject_pool if s != 'English'], 3)), axis=1
    )
    applicants.to_csv('applicants.csv', index=False)
    print("Updated applicants.csv with synthetic UTME subjects")
    
    # Recompute eligibility with UTME validation
    scores = []
    for i, row in applicants.iterrows():
        applicant_id = row['applicant_id']
        utme = row['utme_score']
        subjects = parse_applicant_olevel(row)
        utme_subjects = parse_utme_subjects(row)
        top_interest = row['top_interest']
        related = get_related_courses(top_interest)
        for course in course_names:
            eligible = is_eligible_with_utme(subjects, utme_subjects, course) and utme >= cutoff_marks[course]
            score = 0
            if eligible:
                grade_sum, count = compute_grade_sum(subjects, course)
                interest_match = course == top_interest or course in related
                score = compute_score(utme, grade_sum, count, interest_match, interest_bonus)
            scores.append({'applicant_id': applicant_id, 'course': course, 'eligible': eligible, 'score': score})
    scores_df = pd.DataFrame(scores)
    scores_df.to_csv('scores_utme_validated.csv', index=False)
    print(f"Saved scores_utme_validated.csv with {len(scores_df)} entries")
    display(scores_df.head())
except Exception as e:
    print(f"Error validating UTME subjects: {e}")

In [ ]:
```chartjs
{
  "type": "box",
  "data": {
    "labels": [],
    "datasets": []
  },
  "options": {
    "scales": {
      "y": {
        "beginAtZero": true,
        "title": {
          "display": true,
          "text": "UTME Score"
        }
      },
      "x": {
        "title": {
          "display": true,
          "text": "Course"
        }
      }
    },
    "plugins": {
      "title": {
        "display": true,
        "text": "UTME Score Distribution for Top Courses"
      },
      "legend": {
        "display": false
      }
    }
  }
}
```

# Analyze UTME score distributions for top courses
try:
    if not os.path.exists('assignments.csv') or not os.path.exists('applicants.csv'):
        raise FileNotFoundError("Required files missing. Please run Cells 6 and 8 first.")
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    
    # Define top courses (based on high cut-off marks)
    top_courses = ['Computer Science', 'Human Anatomy', 'Medical Laboratory Science', 'Physiology', 
                   'Civil Engineering', 'Computer Engineering', 'Electrical / Electronics Engineering']
    
    # Merge assignments with UTME scores
    merged_df = assignments_df.merge(applicants_df[['applicant_id', 'utme_score']], on='applicant_id', how='left')
    datasets = []
    labels = []
    
    for course in top_courses:
        scores = merged_df[(merged_df['assigned_program'] == course) & (merged_df['assigned_program'] != 'UNASSIGNED')]['utme_score'].dropna()
        if not scores.empty:
            labels.append(course)
            datasets.append({
                "label": course,
                "data": scores.tolist(),
                "backgroundColor": "rgba(75, 192, 192, 0.5)",
                "borderColor": "rgba(75, 192, 192, 1)",
                "borderWidth": 1
            })
    
    # Update chart data
    chart_data = {
        "type": "box",
        "data": {
            "labels": labels,
            "datasets": datasets
        },
        "options": {
            "scales": {
                "y": {
                    "beginAtZero": true,
                    "title": {
                        "display": true,
                        "text": "UTME Score"
                    }
                },
                "x": {
                    "title": {
                        "display": true,
                        "text": "Course"
                    }
                }
            },
            "plugins": {
                "title": {
                    "display": true,
                    "text": "UTME Score Distribution for Top Courses"
                },
                "legend": {
                    "display": false
                }
            }
        }
    }
    print("Box plot created for UTME score distributions in top courses")
except Exception as e:
    print(f"Error creating score distribution plot: {e}")

In [ ]:
# Import required libraries
import pandas as pd
import os

# Export results to Excel
try:
    if not os.path.exists('assignments.csv') or not os.path.exists('applicants.csv') or not os.path.exists('scores_utme_validated.csv'):
        raise FileNotFoundError("Required files missing. Please run Cells 6, 8, and 15 first.")
    
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    scores_df = pd.read_csv('scores_utme_validated.csv')
    
    # Create Excel writer
    with pd.ExcelWriter('futa_admissions_results.xlsx', engine='openpyxl') as writer:
        # Sheet 1: Allocations
        assignments_df.to_excel(writer, sheet_name='Allocations', index=False)
        
        # Sheet 2: Applicant Details
        applicants_df.to_excel(writer, sheet_name='Applicants', index=False)
        
        # Sheet 3: Scores
        scores_df.to_excel(writer, sheet_name='Scores', index=False)
        
        # Sheet 4: Allocation Summary
        assigned_counts = assignments_df[assignments_df['assigned_program'] != 'UNASSIGNED']['assigned_program'].value_counts().reset_index()
        assigned_counts.columns = ['Course', 'Students Assigned']
        assigned_counts['Capacity'] = assigned_counts['Course'].map(capacities)
        assigned_counts['Utilization (%)'] = assigned_counts.apply(
            lambda x: (x['Students Assigned'] / x['Capacity'] * 100) if x['Capacity'] > 0 else 0, axis=1
        )
        assigned_counts.to_excel(writer, sheet_name='Summary', index=False)
        
        # Sheet 5: Score Statistics for Top Courses
        top_courses = ['Computer Science', 'Human Anatomy', 'Medical Laboratory Science', 'Physiology', 
                       'Civil Engineering', 'Computer Engineering', 'Electrical / Electronics Engineering']
        merged_df = assignments_df.merge(applicants_df[['applicant_id', 'utme_score']], on='applicant_id', how='left')
        stats = []
        for course in top_courses:
            scores = merged_df[(merged_df['assigned_program'] == course) & (merged_df['assigned_program'] != 'UNASSIGNED')]['utme_score'].dropna()
            if not scores.empty:
                stats.append({
                    'Course': course,
                    'Median UTME': scores.median(),
                    'Min UTME': scores.min(),
                    'Max UTME': scores.max(),
                    'Assigned Students': len(scores)
                })
        stats_df = pd.DataFrame(stats)
        stats_df.to_excel(writer, sheet_name='Score_Stats', index=False)
    
    print("Results exported to futa_admissions_results.xlsx")
except Exception as e:
    print(f"Error exporting to Excel: {e}")

In [ ]:
# Generate a project checklist in Markdown
try:
    from datetime import datetime
    checklist = f"""
# FUTA Admissions Placement Model: Project Checklist
*Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*

## Academic Requirements Checklist
- [x] **Model Development**: Implemented a comprehensive admissions model handling all 48 FUTA courses (Cells 0–9).
- [x] **Error Handling**: Included try-except blocks in all cells to handle missing files, invalid data, and other errors.
- [x] **Validation**: Validated courses, capacities, and cut-off marks (Cells 2, 4, 7).
- [x] **Unit Testing**: Ran unit tests to verify parsing, eligibility, and scoring (Cell 10).
- [x] **Course-Specific Cut-Off Marks**: Incorporated higher cut-offs for competitive courses (e.g., 250 for medical, Cell 7).
- [x] **UTME Subject Validation**: Added checks for UTME subject requirements (Cell 15).
- [x] **Analysis**: Analyzed allocation results, including first-choice success rate and capacity utilization (Cell 11).
- [x] **Visualization**: Created a bar chart for assigned students (Cell 12) and a box plot for UTME score distributions (Cell 16).
- [x] **Prediction**: Implemented a function to predict placement for new applicants (Cell 13).
- [x] **Reporting**: Generated a detailed Markdown report (Cell 14) and Excel output (Cell 17).
- [ ] **Real Data Testing**: Replace synthetic data with real applicant data in Cell 6 (if available).
- [ ] **Documentation**: Write a final project report including methodology, results, and conclusions.
- [ ] **Presentation**: Prepare slides or a summary for project defense (if required).

## Generated Files
- [x] courses.csv
- [x] applicants.csv
- [x] scores.csv
- [x] scores_utme_validated.csv
- [x] assignments.csv
- [x] model.joblib
- [x] admissions_report.md
- [x] futa_admissions_results.xlsx

## Next Steps for Submission
1. **Test with Real Data**: Update Cell 6 to load real applicant data (ensure columns: applicant_id, utme_score, top_interest, subject1–subject5, utme_subjects).
2. **Validate Outputs**: Check `scores_utme_validated.csv` and `assignments.csv` to ensure allocations respect cut-off marks and UTME subjects.
3. **Finalize Report**: Expand `admissions_report.md` with methodology details, analysis of results, and limitations.
4. **Prepare Appendices**: Include `futa_admissions_results.xlsx` and sample outputs in your project report.
5. **Review Code**: Ensure all cells have clear comments and are well-documented.
6. **Submit Notebook**: Save `futa_admissions.ipynb` with all outputs visible (run all cells before saving).

*End of Checklist*
"""
    
    # Save checklist to file
    with open('project_checklist.md', 'w') as f:
        f.write(checklist)
    print("Project checklist saved as project_checklist.md")
    
    # Display checklist in notebook
    from IPython.display import Markdown
    display(Markdown(checklist))
except Exception as e:
    print(f"Error generating project checklist: {e}")

In [ ]:
# Import required libraries
import pandas as pd
import os

# Generate merit lists for each course
try:
    if not os.path.exists('assignments.csv') or not os.path.exists('applicants.csv') or not os.path.exists('scores_utme_validated.csv'):
        raise FileNotFoundError("Required files missing. Please run Cells 6, 8, and 15 first.")
    
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    scores_df = pd.read_csv('scores_utme_validated.csv')
    
    # Merge data for merit lists
    merged_df = assignments_df.merge(applicants_df[['applicant_id', 'utme_score', 'top_interest', 'subject1', 'subject2', 'subject3', 'subject4', 'subject5']], 
                                    on='applicant_id', how='left')
    merged_df = merged_df.merge(scores_df, on=['applicant_id', 'assigned_program'], how='left', suffixes=('', '_score'))
    
    # Filter assigned applicants and rename columns for clarity
    merit_df = merged_df[merged_df['assigned_program'] != 'UNASSIGNED'][['applicant_id', 'assigned_program', 'score', 'utme_score', 
                                                                       'subject1', 'subject2', 'subject3', 'subject4', 'subject5', 
                                                                       'assigned_rank', 'reason']]
    merit_df = merit_df.rename(columns={'assigned_program': 'Course', 'score': 'Composite Score', 'utme_score': 'UTME Score', 
                                        'assigned_rank': 'Rank'})
    
    # Create merit lists for each course
    with pd.ExcelWriter('merit_lists.xlsx', engine='openpyxl') as writer:
        for course in course_names:
            course_merit = merit_df[merit_df['Course'] == course].sort_values(by=['Composite Score', 'UTME Score'], ascending=[False, False])
            if not course_merit.empty:
                course_merit.to_excel(writer, sheet_name=course[:31], index=False)  # Excel sheet names limited to 31 characters
        print("Merit lists saved to merit_lists.xlsx")
    
    # Display sample merit list for a top course (e.g., Computer Science)
    sample_merit = merit_df[merit_df['Course'] == 'Computer Science'].head()
    print("\nSample Merit List for Computer Science:")
    display(sample_merit)
except Exception as e:
    print(f"Error generating merit lists: {e}")

In [ ]:
# Import required libraries
import pandas as pd
import os

# Analyze fairness across interest groups
try:
    if not os.path.exists('assignments.csv') or not os.path.exists('applicants.csv'):
        raise FileNotFoundError("Required files missing. Please run Cells 6 and 8 first.")
    
    assignments_df = pd.read_csv('assignments.csv')
    applicants_df = pd.read_csv('applicants.csv')
    
    # Merge assignments with top interests
    merged_df = assignments_df.merge(applicants_df[['applicant_id', 'top_interest']], on='applicant_id', how='left')
    
    # Map courses to interest groups
    def get_interest_group(course):
        for group, courses in course_groups.items():
            if course in courses:
                return group
        return 'other'
    
    merged_df['interest_group'] = merged_df['top_interest'].apply(get_interest_group)
    merged_df['assigned_group'] = merged_df['assigned_program'].apply(get_interest_group)
    
    # Compute fairness metrics
    fairness_stats = []
    for group in course_groups.keys():
        group_applicants = merged_df[merged_df['interest_group'] == group]
        total_applicants = len(group_applicants)
        assigned_same_group = len(group_applicants[group_applicants['assigned_group'] == group])
        assigned_any = len(group_applicants[group_applicants['assigned_program'] != 'UNASSIGNED'])
        success_rate_same_group = (assigned_same_group / total_applicants * 100) if total_applicants > 0 else 0
        success_rate_any = (assigned_any / total_applicants * 100) if total_applicants > 0 else 0
        fairness_stats.append({
            'Interest Group': group,
            'Total Applicants': total_applicants,
            'Assigned to Same Group': assigned_same_group,
            'Assigned to Any Program': assigned_any,
            'Same Group Success Rate (%)': success_rate_same_group,
            'Overall Assignment Rate (%)': success_rate_any
        })
    
    fairness_df = pd.DataFrame(fairness_stats)
    fairness_df.to_csv('fairness_analysis.csv', index=False)
    print("Fairness analysis saved to fairness_analysis.csv")
    print("\nFairness Analysis Summary:")
    display(fairness_df)
except Exception as e:
    print(f"Error performing fairness analysis: {e}")

In [ ]:
# Import required libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Interactive prediction interface
try:
    # Define widgets
    utme_score = widgets.IntSlider(value=180, min=0, max=400, step=1, description='UTME Score:')
    top_interest = widgets.Dropdown(options=course_names, description='Top Interest:')
    subject1 = widgets.Dropdown(options=common_subjects, description='Subject 1:')
    grade1 = widgets.Dropdown(options=list(grade_map.keys()), description='Grade 1:')
    subject2 = widgets.Dropdown(options=common_subjects, description='Subject 2:')
    grade2 = widgets.Dropdown(options=list(grade_map.keys()), description='Grade 2:')
    subject3 = widgets.Dropdown(options=common_subjects, description='Subject 3:')
    grade3 = widgets.Dropdown(options=list(grade_map.keys()), description='Grade 3:')
    subject4 = widgets.Dropdown(options=common_subjects, description='Subject 4:')
    grade4 = widgets.Dropdown(options=list(grade_map.keys()), description='Grade 4:')
    subject5 = widgets.Dropdown(options=common_subjects, description='Subject 5:')
    grade5 = widgets.Dropdown(options=list(grade_map.keys()), description='Grade 5:')
    utme_subjects = widgets.Text(value='English,Mathematics,Physics,Chemistry', description='UTME Subjects:', placeholder='e.g., English,Mathematics,Physics,Chemistry')
    predict_button = widgets.Button(description='Predict Placement')
    output = widgets.Output()
    
    # Define prediction callback
    def on_predict_button_clicked(b):
        with output:
            output.clear_output()
            olevel_subjects = {
                subject1.value: grade_map[grade1.value],
                subject2.value: grade_map[grade2.value],
                subject3.value: grade_map[grade3.value],
                subject4.value: grade_map[grade4.value],
                subject5.value: grade_map[grade5.value]
            }
            utme_subjects_list = [s.strip() for s in utme_subjects.value.split(',')]
            
            # Modified predict_placement to include UTME subjects
            try:
                results = []
                related = get_related_courses(top_interest.value)
                for course in course_names:
                    eligible = is_eligible_with_utme(olevel_subjects, utme_subjects_list, course) and utme_score.value >= cutoff_marks[course]
                    score = 0
                    if eligible:
                        grade_sum, count = compute_grade_sum(olevel_subjects, course)
                        interest_match = course == top_interest.value or course in related
                        score = compute_score(utme_score.value, grade_sum, count, interest_match, interest_bonus)
                    results.append({'course': course, 'eligible': eligible, 'score': score})
                
                results_df = pd.DataFrame(results)
                eligible_courses = results_df[results_df['eligible'] & (results_df['score'] > 0)]
                if eligible_courses.empty:
                    print("Predicted Program: UNASSIGNED")
                    print("Score: 0.00")
                    print("Reason: No eligible course")
                else:
                    best_course = eligible_courses.loc[eligible_courses['score'].idxmax()]
                    print(f"Predicted Program: {best_course['course']}")
                    print(f"Score: {best_course['score']:.2f}")
                    print(f"Reason: {'highest_score' if best_course['course'] == top_interest.value else 'highest_score_alternative'}")
            except Exception as e:
                print(f"Error in prediction: {e}")
    
    predict_button.on_click(on_predict_button_clicked)
    
    # Display interface
    display(widgets.VBox([
        utme_score, top_interest, 
        widgets.HBox([subject1, grade1]),
        widgets.HBox([subject2, grade2]),
        widgets.HBox([subject3, grade3]),
        widgets.HBox([subject4, grade4]),
        widgets.HBox([subject5, grade5]),
        utme_subjects, predict_button, output
    ]))
    print("Interactive prediction interface loaded")
except Exception as e:
    print(f"Error creating interactive interface: {e}")

In [ ]:
# Import required libraries
import os
import zipfile
import pandas as pd
from datetime import datetime

# Final validation and submission package
try:
    # List expected output files
    expected_files = [
        'courses.csv', 'applicants.csv', 'scores_utme_validated.csv', 'assignments.csv',
        'model.joblib', 'admissions_report.md', 'project_checklist.md', 'merit_lists.xlsx',
        'fairness_analysis.csv', 'futa_admissions_results.xlsx'
    ]
    
    # Validate file existence and non-empty status
    for file in expected_files:
        if not os.path.exists(file):
            raise FileNotFoundError(f"Missing file: {file}. Please run relevant cells.")
        if file.endswith('.csv') or file.endswith('.xlsx'):
            if file.endswith('.csv'):
                df = pd.read_csv(file)
            else:
                df = pd.read_excel(file, sheet_name=None)
            if isinstance(df, dict):  # Excel with multiple sheets
                for sheet, sheet_df in df.items():
                    if sheet_df.empty:
                        print(f"Warning: {file} (sheet: {sheet}) is empty")
            elif df.empty:
                print(f"Warning: {file} is empty")
    
    # Validate key metrics
    assignments_df = pd.read_csv('assignments.csv')
    total_assigned = len(assignments_df[assignments_df['assigned_program'] != 'UNASSIGNED'])
    total_applicants = len(pd.read_csv('applicants.csv'))
    if total_assigned > total_applicants:
        raise ValueError("More assignments than applicants detected")
    print(f"Validation: {total_assigned}/{total_applicants} applicants assigned")
    
    # Create submission package
    submission_zip = f"futa_admissions_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
    with zipfile.ZipFile(submission_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file in expected_files:
            if os.path.exists(file):
                zf.write(file, file)
        zf.write('futa_admissions.ipynb', 'futa_admissions.ipynb')
    print(f"Submission package created: {submission_zip}")
    
    # Optional: Clean up temporary files (uncomment to enable)
    # for file in ['scores.csv']:  # Only clean up non-essential files
    #     if os.path.exists(file):
    #         os.remove(file)
    #         print(f"Deleted temporary file: {file}")
    
    print("Final validation completed successfully")
except Exception as e:
    print(f"Error in final validation: {e}")